In [1]:
#include <iostream>

#include "setup.h"
#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"

using namespace operations_research;

const double infinity = MPSolver::infinity();

In [2]:
{
    MPSolver solver("simple_scip_program", MPSolver::SCIP_MIXED_INTEGER_PROGRAMMING);

    MPVariable* const x = solver.MakeIntVar(0.0, infinity, "x");
    MPVariable* const y = solver.MakeIntVar(0.0, infinity, "y");

    std::cout << "Number of variables = " << solver.NumVariables();
    
    // x + 7 * y <= 17.5.
    MPConstraint* const c0 = solver.MakeRowConstraint(-infinity, 17.5, "c0");
    c0->SetCoefficient(x, 1);
    c0->SetCoefficient(y, 7);

    // x <= 3.5.
    MPConstraint* const c1 = solver.MakeRowConstraint(-infinity, 3.5, "c1");
    c1->SetCoefficient(x, 1);
    c1->SetCoefficient(y, 0);

    std::cout << "Number of constraints = " << solver.NumConstraints();
    
    // Maximize x + 10 * y.
    MPObjective* const objective = solver.MutableObjective();
    objective->SetCoefficient(x, 1);
    objective->SetCoefficient(y, 10);
    objective->SetMaximization();
    
    const MPSolver::ResultStatus result_status = solver.Solve();
    // Check that the problem has an optimal solution.
    if (result_status != MPSolver::OPTIMAL) {
         std::cout << "The problem does not have an optimal solution!";
    }
    
    std::cout << "Solution:" << std::endl;
    std::cout << "Objective value = " << objective->Value() << std::endl;
    std::cout << "x = " << x->solution_value() << std::endl;
    std::cout << "y = " << y->solution_value() << std::endl;

    std::cout << "\nAdvanced usage:" << std::endl;
    std::cout << "Problem solved in " << solver.wall_time() << " milliseconds" << std::endl;
    std::cout << "Problem solved in " << solver.iterations() << " iterations" << std::endl;
    std::cout << "Problem solved in " << solver.nodes() << " branch-and-bound nodes" << std::endl;
}

Number of variables = 2Number of constraints = 2Solution:
Objective value = 23
x = 3
y = 2

Advanced usage:
Problem solved in 139 milliseconds
Problem solved in 0 iterations
Problem solved in 1 branch-and-bound nodes


In [3]:
{
    MPModelProto model_proto;
    model_proto.set_name("Max_Example");

    // Create variables and objective function
    MPVariableProto* x = model_proto.add_variable();
    x->set_name("x");  
    x->set_lower_bound(0.0);
    x->set_upper_bound(infinity);  
    x->set_is_integer(true);      
    x->set_objective_coefficient(1.0);
    
    MPVariableProto* y = model_proto.add_variable();
    y->set_name("y");  
    y->set_lower_bound(0.0);
    y->set_upper_bound(infinity);  
    y->set_is_integer(true);      
    y->set_objective_coefficient(10.0);
    
    // Maximize x + 10 * y.
    model_proto.set_maximize(true);

    // Create constraints
    // x + 7 * y <= 17.5.
    MPConstraintProto* c0 = model_proto.add_constraint();
    c0->set_name("c0");  
    c0->set_lower_bound(-infinity);    
    c0->set_upper_bound(17.5);
    c0->add_var_index(0);
    c0->add_coefficient(1);
    c0->add_var_index(1);
    c0->add_coefficient(7);

    // x <= 3.5.
    MPConstraintProto* c1 = model_proto.add_constraint();
    c1->set_name("c1");  
    c1->set_lower_bound(-infinity);    
    c1->set_upper_bound(3.5);
    c1->add_var_index(0);
    c1->add_coefficient(1);

    MPModelRequest model_request;
    *model_request.mutable_model() = model_proto;
    model_request.set_solver_type(MPModelRequest::SCIP_MIXED_INTEGER_PROGRAMMING);

    MPSolutionResponse solution_response;
    MPSolver::SolveWithProto(model_request, &solution_response);

    // The problem has an optimal solution.
    CHECK_EQ(MPSOLVER_OPTIMAL, solution_response.status());

    std::cout << "objective = " << solution_response.objective_value() << std::endl;
    for (int j = 0; j < 2; ++j) {
    std::cout << model_proto.variable(j).name() << " = "
              << solution_response.variable_value(j) << std::endl;
    }
}

objective = 23
x = 3
y = 2
